In [ ]:
# imports --> To do --> get comments of each one of them, load them when they are going to be used?
import tensorflow as tf
from tensorflow.contrib.slim.nets import inception
import tensorflow.contrib.slim as slim
import tensorflow.python.platform
from tensorflow.python.platform import gfile
import os
import scipy
import sys
import re

# general, prepocessing
import numpy as np

# preprocessing
from PIL import Image

# check them out
import os
import re
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import pandas as pd

#some errors
# import sklearn
# from sklearn import cross_validation
# from sklearn.metrics import accuracy_score, confusion_matrix
# from sklearn.svm import SVC, LinearSVC
# import skimage.io
# import skimage.transform
# from scipy.misc import imresize
# from skimage.transform import resize #al final puede q no se use

# architecture paths
ARCHITECTURES = "pretrained-model/"
INCEPTION_V3_PATH = "pretrained-model/inception_v3.ckpt"
INCEPTION_V4_PATH = "pretrained-model/inception_v4.ckpt"

# image dataset path
PATH_TO_IMG = "../img-raw"

# img data --> To do --> get automated
WIDTH = 299
HEIGHT = 299
CHANNELS = 3

# CONST --> To do --> get automated
N_IMG = 5000 # number of imgs
N_FEATURES =  2048 # number of deepfeatures

In [ ]:
# https://github.com/taey16/tf/tree/master/imagenet --> classify_image_graph_def.pb
def create_graph():
    with gfile.FastGFile(os.path.join(ARCHITECTURES, 'classify_image_graph_def.pb'), 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')

# extract all features from pool layer of InceptionV3
def extract_features(imgList):
    nb_features = 2048
    features = np.empty((len(imgList),nb_features))
    labels = []

    create_graph()

    with tf.Session() as sess:

        next_to_last_tensor = sess.graph.get_tensor_by_name('pool_3:0')

        for ind, image in enumerate(imgList):
#             if (ind%100 == 0):
#                 print('Processing %s...' % (image))
    
#             if not gfile.Exists(image):
#                 tf.logging.fatal('File does not exist %s', image)

            image_data = gfile.FastGFile(image, 'rb').read()
            predictions = sess.run(next_to_last_tensor, {'DecodeJpeg/contents:0': image_data})
            features[ind,:] = np.squeeze(predictions)
            labels.append(re.split('_\d+',image.split('/')[1])[0])

    return features, labels

#function call
features,labels = extract_features(imgList)

#save info to pickle
pickle.dump(features, open('features', 'wb'))
pickle.dump(labels, open('labels', 'wb'))

features = pickle.load(open('features'))
labels = pickle.load(open('labels'))